In [ ]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%config IPCompleter.greedy=True

In [ ]:
sns.set_style('dark')
plt.rcParams['figure.figsize'] = 20, 10
plt.rcParams['lines.markeredgewidth'] = 1
plt.style.use("dark_background")


In [ ]:
current_palette = sns.color_palette()
sns.palplot(current_palette)

In [ ]:
# loading the input data

xl_file_sociale_zekerheid = pd.ExcelFile('data/CBS/Tijdreeksen_sociale_zekerheid.xlsx')
df_sociale_zekerheid = xl_file_sociale_zekerheid.parse(xl_file_sociale_zekerheid.sheet_names[0])
xl_file_bevolking_groei = pd.ExcelFile('data/CBS/Bevolking__huishoudens_en_bevolkingsontwikkeling.xlsx')
df_bevolking_groei = xl_file_bevolking_groei.parse(xl_file_bevolking_groei.sheet_names[0])
xl_file_arbeids_deelname = pd.ExcelFile('data/CBS/Arbeidsdeelname__vanaf_1969.xlsx')
df_arbeids_deelname = xl_file_arbeids_deelname.parse(xl_file_arbeids_deelname.sheet_names[0])

df_arbeids_deelname['Werkloosheidspercentage'] = df_arbeids_deelname['Werkloosheidspercentage'].apply(lambda x: float(x.replace(',', '.')))

In [ ]:
# couple of constants
START_JAAR = 1969

In [ ]:
# filter all data to after pensions were made law
df_sociale_zekerheid = df_sociale_zekerheid.set_index('Perioden').loc[df_sociale_zekerheid.set_index('Perioden').index >= START_JAAR]
df_bevolking_groei = df_bevolking_groei.set_index('Perioden').T.loc[df_bevolking_groei.set_index('Perioden').T.index >= START_JAAR]
df_arbeids_deelname = df_arbeids_deelname.set_index('Perioden').loc[df_arbeids_deelname.set_index('Perioden').index >= START_JAAR]

In [ ]:
ax = df_bevolking_groei['Totaal bevolking'].plot(kind='bar')
ax.set_title('Bevolkingsgroei vanaf 1969', size=20)
ax.set_xlabel('Jaar', size=16)
ax.set_ylabel('Bevolking (x 1000)', size=16)
ax.get_figure().savefig('output/bevolkingsgroei.svg', bbox_inches = 'tight')

In [ ]:
print("van {} naar {} (x 1000)".format(df_bevolking_groei['Totaal bevolking'].iloc[0], df_bevolking_groei['Totaal bevolking'].iloc[-1]))

In [ ]:
df1 = pd.DataFrame()
df1['0-65'] = df_bevolking_groei['Totaal bevolking'] - df_bevolking_groei['65 tot 80 jaar'] - df_bevolking_groei['80 jaar of ouder']
df1['65+'] = df_bevolking_groei['65 tot 80 jaar'] + df_bevolking_groei['80 jaar of ouder']
df1['percentage 65+'] = df1['65+']/(df1['0-65']+df1['65+'])*100
df1['percentage 65+'] = df1['percentage 65+'].apply(lambda x: round(x, 2))

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2)
fig.suptitle('Aandeel 65+ in totale bevolking', size=20)
df1[['0-65', '65+']].plot(kind='bar', stacked=True, ax=axes[0])
axes[0].set_xlabel('Jaar', size=16)
axes[0].set_ylabel('Aantal (x 1000)', size=16)
df1[['percentage 65+']].plot(kind='line', ax=axes[1], grid=True, marker='*')
axes[1].set_xlabel('Jaar', size=16)
axes[1].set_ylabel('Percentage (%)', size=16)
axes[1].set_ylim(9,20)
axes[1].set_xlim(1965, 2020)
axes[1].get_legend().remove()
fig.savefig('output/aandeel65.svg', bbox_inches = 'tight')

In [ ]:
print("aantal van {} naar {} (x1000)".format(df1['65+'].iloc[0], df1['65+'].iloc[-1]))
print("percentage van {} naar {}".format(df1['percentage 65+'].iloc[0], df1['percentage 65+'].iloc[-1]))

In [ ]:
df2 = pd.DataFrame()

df2['Overige bevolking'] = df_bevolking_groei['Totaal bevolking'] - df_arbeids_deelname['Beroepsbevolking']
df2['Beroepsbevolking'] = df_arbeids_deelname['Beroepsbevolking']
df2['Beroepsbevolking (%)'] = df2['Beroepsbevolking']/(df2['Beroepsbevolking'] + df2['Overige bevolking']) * 100
df2 = df2[:-1] #remove 2018 with incomplete data
df2['Beroepsbevolking (%)'] = df2['Beroepsbevolking (%)'].apply(lambda x: round(x, 2))

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2)
fig.suptitle('Aandeel beroepsbevolking in totale bevolking', size=20)
df2[['Overige bevolking', 'Beroepsbevolking']].plot(kind='bar', stacked=True, ax=axes[0])
axes[0].set_xlabel('Jaar', size=16)
axes[0].set_ylabel('Aantal (x 1000)', size=16)
df2[['Beroepsbevolking (%)']].plot(kind='line', ax=axes[1], grid=True, marker='*')
axes[1].set_xlabel('Jaar', size=16)
axes[1].set_ylabel('Percentage (%)', size=16)
axes[1].set_ylim(40,60)
axes[1].set_xlim(1965, 2020)
axes[1].get_legend().remove()
fig.savefig('output/aandeelberoepsbevolking.svg', bbox_inches = 'tight')

In [ ]:
print("beroepsbevolking van {} naar {} (x 1000)".format(df2['Beroepsbevolking'].iloc[0], df2['Beroepsbevolking'].iloc[-1]))
print("percentage beroepsbevolking van {} naar {} (%)".format(df2['Beroepsbevolking (%)'].iloc[0],df2['Beroepsbevolking (%)'].iloc[-1]))

In [ ]:
df3 = pd.DataFrame()
df3['Beroepsbevolking'] = df_arbeids_deelname['Beroepsbevolking']
df3['AOW'] = df_sociale_zekerheid['AOW']
df3 = df3[:-1] #remove 2018 with incomplete data
df3['Werkende per AOW'] = df3['Beroepsbevolking'] / df3['AOW']
df3['Werkende per AOW'] = df3['Werkende per AOW'].apply(lambda x: round(x, 2))

Belangrijke wetswijzigingen: Per 1 april 1985 heeft in de AOW, een algemene volksverzekering voor een welvaartsvast ouderdomspensioen bij het bereiken van de 65-jarige leeftijd, een aanpassing plaatsgehad conform de EG-richtlijnen over de gelijke behandeling van man en vrouw. De gehuwde man en vrouw hebben sindsdien ieder een zelfstandig recht op ouderdomspensioen als zij 65 jaar of ouder zijn. Het pensioen is gesteld op 50% van het netto-minimumloon. In de jaren voor 1985 werd onderscheid gemaakt tussen een uitkering voor gehuwden en een uitkering voor ongehuwden. De cijfers zijn vanaf dat moment daarom niet zonder meer vergelijkbaar met die van voorgaande jaren. Vanaf 1 januari 2013 wordt de AOW-leeftijd jaarljiks met een of meerdere maanden verhoogd en wordt op termijn gekoppeld aan de levensverwachting. De aantallen geven de situatie weer op 31 december.

In [ ]:
ax = df_sociale_zekerheid['AOW'].plot(grid=True, marker='*')
ax.set_xlim(1965, 2020)
ax.set_ylim(1000, 3500)
ax.set_title ('Aantal AOW uitkeringen', size=20)
ax.set_xlabel('Jaar', size=16)
ax.set_ylabel('Aantal (x 1000)', size=16)
ax.get_figure().savefig('output/aantaalaowuitkeringen.svg', bbox_inches = 'tight')

In [ ]:
ax = df3['Werkende per AOW'].plot(grid=True, marker='*')
ax.set_ylim(2.5, 5.5)
ax.set_xlim(1965, 2020)
ax.set_title ('Aantal werkende per AOW uitkering', size=20)
ax.set_xlabel('Jaar', size=16)
ax.set_ylabel('Aantal', size=16);
ax.get_figure().savefig('output/aantalwerkendeperaow.svg', bbox_inches = 'tight')

In [ ]:
df4 = pd.DataFrame(columns=["tijdsperiode", "voor", "na"], data=[
    ["1969-1985", df3['Werkende per AOW'][1969], df3['Werkende per AOW'][1984]],
    ["1986-2017", df3['Werkende per AOW'][1986], df3['Werkende per AOW'].iloc[-1]]
])
df4["verschil (%)"] = round((df4['na']-df4['voor'])/df4['na']*100,2)
df4 = df4.set_index('tijdsperiode')

In [ ]:
df4